<a href="https://colab.research.google.com/github/HerianRoyH/yellow_taxi_trip_project/blob/main/extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import pandas_gbq
#!pip install pandas-gbq

In [2]:
def extract_latest_link(url):
    """Extracts the latest link containing yellow_tripdata from the given URL.

    Args:
        url: The URL to scrape for links.

    Returns:
        The latest link containing yellow_tripdata, or None if no such link is found.
    """
    try:
        page = requests.get(url)
        page.raise_for_status()  # Raise exception for bad status codes

        soup = BeautifulSoup(page.content, 'html.parser')

        links = soup.find_all('a', href=re.compile(r'yellow_tripdata'))

        latest_date = None
        latest_link = None

        for link in links:
            href = link['href']
            date_match = re.search(r'\d{4}-\d{2}', href)
            if date_match:
                date_str = date_match.group()
                if latest_date is None or date_str > latest_date:
                    latest_date = date_str
                    latest_link = href

        return latest_link

    except requests.exceptions.RequestException as e:
        print(f"Error: An error occurred while extracting the latest link - {e}")
        return None



def download_and_load_data(url, filename):
  """Downloads a file from the specified URL, saves it, and loads it as a Pandas DataFrame.

  Args:
      url: The URL of the file to download.
      filename: The name of the file to save the downloaded data to.

  Returns:
      A Pandas DataFrame containing the downloaded data (if successful), or None if download or loading fails.
  """
  try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise exception for bad status codes

    # Handle potential issues with response content type
    if 'content-type' not in response.headers or not response.headers['content-type'].startswith('application/octet-stream'):
      print(f"Warning: Unexpected content type: {response.headers['content-type']}")

    with open(filename, 'wb') as f:
      for chunk in response.iter_content(1024):
        f.write(chunk)

    # Assuming the file is Parquet format, load it into a DataFrame using pandas
    df = pd.read_parquet(filename)

    return df

  except requests.exceptions.RequestException as e:
    print(f"Error: An error occurred while downloading or loading data - {e}")
    return None

In [3]:
page_url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'

latest_link = extract_latest_link(page_url).strip()

file_link = latest_link.split('/')[-1].strip()


In [4]:
#Usage
url = latest_link
filename = file_link

data_df = download_and_load_data(url, filename)

if data_df is not None:
  print("Data downloaded and loaded successfully!")
  print(data_df.head(5))
else:
  print("Download or loading failed.")


Data downloaded and loaded successfully!
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-02-01 00:04:45   2024-02-01 00:19:58              1.0   
1         2  2024-02-01 00:56:31   2024-02-01 01:10:53              1.0   
2         2  2024-02-01 00:07:50   2024-02-01 00:43:12              2.0   
3         1  2024-02-01 00:01:49   2024-02-01 00:10:47              1.0   
4         1  2024-02-01 00:37:35   2024-02-01 00:51:15              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           4.39         1.0                  N            68           236   
1           7.71         1.0                  N            48           243   
2          28.69         2.0                  N           132           261   
3           1.10         1.0                  N           161           163   
4           2.60         1.0                  N           246            79   

   payment_type  fare_amount  ext

In [5]:
data_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-02-01 00:04:45,2024-02-01 00:19:58,1.0,4.39,1.0,N,68,236,1,20.5,1.0,0.5,1.28,0.00,1.0,26.78,2.5,0.00
1,2,2024-02-01 00:56:31,2024-02-01 01:10:53,1.0,7.71,1.0,N,48,243,1,31.0,1.0,0.5,9.00,0.00,1.0,45.00,2.5,0.00
2,2,2024-02-01 00:07:50,2024-02-01 00:43:12,2.0,28.69,2.0,N,132,261,2,70.0,0.0,0.5,0.00,6.94,1.0,82.69,2.5,1.75
3,1,2024-02-01 00:01:49,2024-02-01 00:10:47,1.0,1.10,1.0,N,161,163,1,9.3,3.5,0.5,2.85,0.00,1.0,17.15,2.5,0.00
4,1,2024-02-01 00:37:35,2024-02-01 00:51:15,1.0,2.60,1.0,N,246,79,2,15.6,3.5,0.5,0.00,0.00,1.0,20.60,2.5,0.00


Extracting February 2024 records.

In [21]:
# Convert 'tpep_pickup_datetime' column to datetime format
data_df['tpep_pickup_datetime'] = pd.to_datetime(data_df['tpep_pickup_datetime'])

# Filter records where passenger_count is greater than 0
filtered_passenger_count = data_df[data_df['passenger_count'] > 0]

# Filter records for the specified date range (from 2024-02-01 to 2024-02-29)
start_date = pd.Timestamp(2024, 2, 1)
end_date = pd.Timestamp(2024, 2, 29)

filtered_data = filtered_passenger_count[
    (filtered_passenger_count['tpep_pickup_datetime'] >= start_date) &
    (filtered_passenger_count['tpep_pickup_datetime'] <= end_date)
]

filtered_data


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-02-01 00:04:45,2024-02-01 00:19:58,1.0,4.39,1.0,N,68,236,1,20.5,1.0,0.5,1.28,0.00,1.0,26.78,2.5,0.00
1,2,2024-02-01 00:56:31,2024-02-01 01:10:53,1.0,7.71,1.0,N,48,243,1,31.0,1.0,0.5,9.00,0.00,1.0,45.00,2.5,0.00
2,2,2024-02-01 00:07:50,2024-02-01 00:43:12,2.0,28.69,2.0,N,132,261,2,70.0,0.0,0.5,0.00,6.94,1.0,82.69,2.5,1.75
3,1,2024-02-01 00:01:49,2024-02-01 00:10:47,1.0,1.10,1.0,N,161,163,1,9.3,3.5,0.5,2.85,0.00,1.0,17.15,2.5,0.00
4,1,2024-02-01 00:37:35,2024-02-01 00:51:15,1.0,2.60,1.0,N,246,79,2,15.6,3.5,0.5,0.00,0.00,1.0,20.60,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706634,2,2024-02-28 23:59:46,2024-02-29 00:13:52,1.0,1.94,1.0,N,230,233,1,14.2,1.0,0.5,4.80,0.00,1.0,24.00,2.5,0.00
2706706,2,2024-02-28 23:53:38,2024-02-29 00:02:20,2.0,1.34,1.0,N,239,143,1,10.0,1.0,0.5,3.00,0.00,1.0,18.00,2.5,0.00
2706793,2,2024-02-28 23:53:55,2024-02-29 00:16:46,5.0,9.33,1.0,N,138,230,1,38.0,6.0,0.5,11.34,6.94,1.0,68.03,2.5,1.75
2775327,2,2024-02-29 00:00:00,2024-02-29 17:57:41,2.0,1.45,1.0,N,186,48,2,10.0,2.5,0.5,0.00,0.00,1.0,16.50,2.5,0.00


In [23]:
table_id = 'yellow-taxi-trip-project.all_raw.yellow_taxi_raw'

# Write the DataFrame to BigQuery
pandas_gbq.to_gbq(filtered_data, table_id, project_id='yellow-taxi-trip-project', if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 5384.22it/s]
